## Step 1: create cultivar specific files

In [51]:
##split the 0_phenotype.txt into cultivar-specific file

import pandas as pd

# Load the file
df = pd.read_csv('0_phenotype_data.txt', sep='\t')

# Splitting the file
output_files = []
for i in range(1, len(df.columns)):
    # Selecting ID column and the ith column
    col_name = df.columns[i]
    subset_df = df.iloc[:, [0, i]]
    
    # Saving to a new file
    output_file_name = f'1_data{i}.txt'
    subset_df.to_csv(output_file_name, sep='\t', index=False)
    output_files.append(output_file_name)

output_files

['1_data1.txt', '1_data2.txt']

## Step 2: change second col header to disease [which was cultivars name]

In [52]:
import pandas as pd
import glob

# Using glob to find all files starting with '1_data' in the specified directory
file_paths = glob.glob('1_data*.txt')

# Renaming the second column in each file
for file_path in file_paths:
    df = pd.read_csv(file_path, sep='\t')
    df.columns = ['ID', 'disease']
    
    # Save the modified file
    df.to_csv(file_path, sep='\t', index=False)


## Step 3: calculate median and assign disease to low if value lower than median, and high - if value higher than median

In [53]:
import pandas as pd
import glob

# Using glob to find all files starting with '1_data' in the specified directory
data_files = glob.glob('1_data*.txt')

# Processing each file
for data_file in data_files:
    data_df = pd.read_csv(data_file, sep='\t')

    # Check if 'disease' column exists

    
    if 'disease' in data_df.columns:
        # Calculate the median
        median_value = data_df['disease'].median()

        # Replace values based on the median
        data_df['disease'] = data_df['disease'].apply(lambda x: 'low' if x < median_value else 'high')

        # Save the modified dataframe back to the file
        data_df.to_csv(data_file, sep='\t', index=False)
    else:
        print(f"'disease' column not found in {data_file}")


## Step 4: process the complete isoform table - remove isoform frequency <5

In [54]:
##check number of isoform in bash
#awk -F'\t' '{print NF; exit}' isoform_output_combined_zymo.txt

In [55]:
## removeing all the isoform with freq <5
# this step can be resourse intensive, so better in nimbus------------------------
import pandas as pd

# Load the file into a pandas DataFrame
df = pd.read_csv('0_combined_isoform.txt', sep='\t')

# Sum the frequencies of each isoform across all samples
isoform_sums = df.iloc[:, 1:].sum()

# Filter out isoforms with a sum of 3 or less
filtered_isoforms = isoform_sums[isoform_sums > 5]

# Creating a new DataFrame with only the filtered isoforms
filtered_df = df[['ID'] + list(filtered_isoforms.index)]

# Saving the filtered DataFrame to a new file
filtered_df.to_csv('0_filtered_combined_isoform.txt', index=False, sep='\t')


## step 5: merge cultivar-specific file with combine isoform file by ID

In [56]:
import pandas as pd
import glob
import os


# File to be concatenated
additional_file = '0_filtered_combined_isoform.txt'

# Read the additional file
additional_df = pd.read_csv(additional_file, sep='\t')

# Using glob to find all files starting with '1_data' in the specified directory
data_files = glob.glob( '1_data*.txt')

# Concatenating each file with the additional file
for data_file in data_files:
    # Read the data file
    data_df = pd.read_csv(data_file, sep='\t')

    # Concatenate with the additional dataframe on 'ID' column
    concatenated_df = pd.merge(data_df, additional_df, on='ID', how='left')

    # Corrected extraction of file number
    file_number = os.path.basename(data_file).replace('1_data', '').replace('.txt', '')
    
    # Save the concatenated dataframe to a new file
    concatenated_file_path =  f'2_merged_data{file_number}.txt'
    concatenated_df.to_csv(concatenated_file_path, sep='\t', index=False)

    print(f"Concatenated file saved: {concatenated_file_path}")



Concatenated file saved: 2_merged_data1.txt
Concatenated file saved: 2_merged_data2.txt


## Step 6: change 0 = A and 1 = P in those files

In [57]:
import pandas as pd
import glob

# Using glob to find all files starting with '1_data' in the specified directory
data_files = glob.glob('2_merged_data*.txt')

# Processing each file
for data_file in data_files:
    data_df = pd.read_csv(data_file, sep='\t')

    # Looping through all columns after the second one
    for col in data_df.columns[2:]:
        # Replace 1 with 'P' and 0 with 'A'
        data_df[col] = data_df[col].apply(lambda x: 'P' if x == 1.0 else ('A' if x == 0.0 else x))

    # Save the modified dataframe back to the file
    data_df.to_csv(data_file, sep='\t', index=False)


## step 7: contingency table

In [58]:
###step 2: contingency table 

import pandas as pd

for i in range(1, 3):  # Loop from 1 to 12 for your 12 data files----------------------------------neeed to be automated here 
    # Dynamically create file name
    file_name = f'2_merged_data{i}.txt'

    # Load data
    data = pd.read_csv(file_name, delimiter='\t')  # Adjust delimiter if needed

    # Melt the data
    melted = pd.melt(data, id_vars=['disease'], var_name='isoform', value_name='value')

    # Pivot the table to get the contingency table
    pivot_data = melted.pivot_table(index='isoform', columns=['disease', 'value'], aggfunc='size', fill_value=0)

    # Create new headers
    pivot_data.columns = ['{}-{}'.format(disease, value) for disease, value in pivot_data.columns]

    # Filter columns
    desired_columns = ['high-A', 'high-P', 'low-A', 'low-P']
    pivot_data = pivot_data[desired_columns]

    # Remove the first row (which contains 'ID')
    pivot_data = pivot_data.drop(pivot_data.index[0])

    # Write the contingency table to a file
    contingency_table_file = f'3_contingency_table_data{i}.txt'
    pivot_data.to_csv(contingency_table_file, sep='\t', index=True)

    # Rename the columns
    column_rename = {'high-A': 'c', 'high-P': 'a', 'low-A': 'd', 'low-P': 'b'}
    pivot_data.rename(columns=column_rename, inplace=True)

    # Write the reformatted contingency table to a new file
    hypergeo_data_file = f'4_hypergeo_data{i}.txt'
    pivot_data.to_csv(hypergeo_data_file, sep='\t', index=True)

    print(f"Script completed for dataset {i}. The formatted contingency table has been written to '{contingency_table_file}' and '{hypergeo_data_file}'")

Script completed for dataset 1. The formatted contingency table has been written to '3_contingency_table_data1.txt' and '4_hypergeo_data1.txt'
Script completed for dataset 2. The formatted contingency table has been written to '3_contingency_table_data2.txt' and '4_hypergeo_data2.txt'


## Step 8: hypergeomatric test

In [59]:
###step 3 - automated: Hypergeo test - loop over many datasets together [automated]
###loop over all dataset
import math

# Factorial calculation function
def calculate_factorials(n):
    fact = [0] * (n + 1)
    fact[0] = 1
    for i in range(2, n + 1):
        fact[i] = fact[i - 1] + math.log(i)
    return fact

def process_dataset(input_file, output_file, fact):
    with open(output_file, 'w') as fileout, open(input_file, 'r') as input_data:
        fileout.write("isoform\thigh-A\thigh-P\tlow-A\tlow-P\tp-value\n")
        next(input_data)  # Skip the header row
        for line_num, line in enumerate(input_data, 2):
            tabdata = line.strip().split('\t')
            try:
                iso = tabdata[0]
                a = round(float(tabdata[1]))
                c = round(float(tabdata[2]))
                b = round(float(tabdata[3]))
                d = round(float(tabdata[4]))
            except ValueError as e:
                print(f"Error on line {line_num}: {e}")
                continue
            
            n = a + b + c + d
            hyp1 = fact[a + b] + fact[c + d] + fact[a + c] + fact[b + d] - (fact[n] + fact[a] + fact[b] + fact[c] + fact[d])
            hyp1 = math.exp(hyp1)
            fileout.write(f"{iso}\t{a}\t{c}\t{b}\t{d}\t{hyp1}\n")

# Determine the range of your datasets
start_dataset_number = 1  # start number -------------------------------------------------------need to change here (need to be automated)
end_dataset_number = 2   # end number ---------------------------------------------------------need to change here (need to be automated)

# Pre-calculate factorials
fact = calculate_factorials(1000000)

# Loop over the datasets
for i in range(start_dataset_number, end_dataset_number + 1):
    input_file = f'4_hypergeo_data{i}.txt'
    output_file = f'5_output_hypergeo_data{i}.txt'
    print(f"Processing dataset number {i}...")
    process_dataset(input_file, output_file, fact)

print("All datasets processed.")


Processing dataset number 1...
Processing dataset number 2...
All datasets processed.


## Step 9: merge all the cultivar-specific p-value (master data)

In [60]:
###step 4: merge_pvalue - cultivar specific [automated]

import pandas as pd
import glob

# Specify the pattern for your files
file_pattern = '5_output_hypergeo_data*.txt'
files = glob.glob(file_pattern)

# Initialize an empty DataFrame for the merged data
merged_data = pd.DataFrame()

# Process each file
for file_number, file_path in enumerate(sorted(files), start=1):
    # Read the file, focusing only on the 'isoform' and 'p-value' columns
    data = pd.read_csv(file_path, delimiter='\t', usecols=['isoform', 'p-value'])
    
    # Rename the 'p-value' column to 'p-value-{i}'
    p_value_column = f'p-value-{file_number}'
    data.rename(columns={'p-value': p_value_column}, inplace=True)
    
    # If merged_data is empty, initialize it with the data from the first file
    if merged_data.empty:
        merged_data = data
    else:
        # Merge the current data with the merged_data on 'isoform'
        merged_data = pd.merge(merged_data, data, on='isoform', how='outer')

# Save the merged data to a new file
merged_data.to_csv('6_merged_p-value.txt', sep='\t', index=False)

print("Script completed. Merged data written to 'merged_output_hypergeo_data.txt'")


Script completed. Merged data written to 'merged_output_hypergeo_data.txt'


## Step 10: calculate lowest p-value

In [61]:
# Re-importing pandas and redefining the code after the reset
import pandas as pd

# File path
file_path = '6_merged_p-value.txt'

# Reading the file
df = pd.read_csv(file_path, sep='\t')

# Finding columns that start with 'p-value'
p_value_cols = [col for col in df.columns if col.startswith('p-value')]

# Calculating the lowest p-value and storing it in a new column
df["p-value_lowest"] = df[p_value_cols].min(axis=1)

# Saving the modified dataframe
modified_file_path = '7_merged_lowest_p-value.txt'
df.to_csv(modified_file_path, sep='\t', index=False)

modified_file_path



'7_merged_lowest_p-value.txt'

## Step 11: need to create col locus_id from isoform_id

In [62]:
###preparing cultivar specific fisher p-value for combining

import pandas as pd

# Read the merged output
merged_df = pd.read_csv("7_merged_lowest_p-value.txt", delimiter="\t")

# Duplicate "isoform_id" column for further processing
merged_df["locus_id"] = merged_df["isoform"]

# Use regex to process the "variable" column
merged_df["locus_id"] = merged_df["locus_id"].str.replace(r'(_\d+)$', '', regex=True)

# Use case-insensitive regex to remove A, B, C, D, or any combination thereof from the end
merged_df["locus_id"] = merged_df["locus_id"].str.replace(r'(?i)([A-D]+)$', '', regex=True)

# Move "variable" column to the first position
cols = ['locus_id'] + [col for col in merged_df if col != 'locus_id']
merged_df = merged_df[cols]

# Save the updated dataframe
merged_df.to_csv("8_merged_lowest_p-value_with_locus_id.txt", index=False, sep="\t")

## Step 12: combine fisher result with predector result

In [63]:
import pandas as pd

# Load the data files
df1 = pd.read_csv("8_merged_lowest_p-value_with_locus_id.txt", sep="\t")
df2 = pd.read_csv("0_predector_results.txt", sep="\t")

# Ensure that 'locus_id' is the first column in both dataframes
df1 = df1[['locus_id'] + [col for col in df1.columns if col != 'locus_id']]
df2 = df2[['locus_id'] + [col for col in df2.columns if col != 'locus_id']]

# Merge the dataframes on 'locus_id'
merged_df = df1.merge(df2, on="locus_id", how="left")

# Fill NA values if 'locus_id' from df1 is not found in df2
merged_df.fillna('NA', inplace=True)

# Save the merged data to a new file
merged_df.to_csv("10_pred_fisher_merged_dataset.txt", index=False, sep="\t")

print("Merging complete!")


Merging complete!


## Step 13: calculate predfish value

In [64]:
# Predector - fisher caculation nd filtering 
import csv

# Lists to store the processed data
new_data = []

# Read the content of the original TSV file
with open("10_pred_fisher_merged_dataset.txt", "r") as file: #####change data set here############
    reader = csv.reader(file, delimiter="\t")
    
    # Process header (add new column name)
    header = next(reader)
    header.extend(["convt_p-value","nor_pred_score", "PF_score", "known_effector"])
    new_data.append(header)

    # Index of the p-value column
    p_value_idx = header.index("p-value_lowest")
    pred_score_idx = header.index("effector_score")
    variable_idx = header.index("locus_id")
    cyst_idx = header.index("aa_c_number")
    total_aa_idx = header.index("residue_number")

    # Process data rows
    for row in reader:

        try:
        #    # Check the conditions for filtering row
           #if float(row[cyst_idx]) < 2 or float(row[total_aa_idx]) > 300  or float(row[pred_score_idx]) < 2 or float(row[p_value_idx]) > 0.05: ###------------------need to change the filters here
           if float(row[pred_score_idx]) < 2 :
            #if float(row[total_aa_idx]) > 3000:
                continue  # Skip the row if any condition is met
        except ValueError:  # handle non-numeric values # this is need as there are some NA value, that why the following section did not work
            continue

        # Calculate convt_p-value
        convt_p_value = 1 - float(row[p_value_idx])
    
        # Calculate nor_pred_score
        nor_pred_score = (float(row[pred_score_idx]) - 1.001) / (3.941 - 1.001)
    
        # Calculate PF_score
        PF_score = (nor_pred_score * 0.5) + (convt_p_value * 0.5)
        
        # Determine known_effector based on variable column
 # Determine known_effector based on variable column
        if "SNOO_200780" in row[variable_idx]:
            known_effector = "Tox1"
        elif "SNOO_165710" in row[variable_idx]:
            known_effector = "ToxA"
        elif "SNOO_423420" in row[variable_idx]:
            known_effector = "Tox1-like"
        elif "SNOO_089810" in row[variable_idx]:
            known_effector = "Tox3"
        elif "SNOO_144930" in row[variable_idx]:
            known_effector = "Tox267"
        elif "SNOO_060150" in row[variable_idx]:
            known_effector = "Tox8"
        elif "SNOO_503200" in row[variable_idx]:
            known_effector = "Tox5"
        elif "SNOO_529790" in row[variable_idx]:
            known_effector = "homologue - tox1, tox1-like"
        elif "SNOO_083550" in row[variable_idx]:
            known_effector = "homologue - tox1"
        elif "SNOO_010970" in row[variable_idx]:
            known_effector = "homologue - tox3, tox5, tox267"
        elif "SNOO_064590" in row[variable_idx]:
            known_effector = "Hydrophobin2-MHP1]"
        elif "SNOO_031220" in row[variable_idx]:
            known_effector = "Hydrophobin2-MHP1]"
        elif "SNOO_017620" in row[variable_idx]:
            known_effector = "TUB1"
        elif "SNOO_045580" in row[variable_idx]:
            known_effector = "Actin-cap_B"
        elif "SNOO_011390" in row[variable_idx]:
            known_effector = "Actin-ARP1"
        elif "SNOO_012780" in row[variable_idx]:
            known_effector = "GAPDH"
        else:
            known_effector = ""
        
        # Append new columns to the row
        row.extend([str(convt_p_value), str(nor_pred_score), str(PF_score), known_effector])
        
        new_data.append(row)

# Write the processed data to the same file
with open("11_pred_fisher_test_result.txt", "w", newline='') as outfile: #####change data set here############
    writer = csv.writer(outfile, delimiter="\t")
    writer.writerows(new_data)

print("New columns have been added.")

New columns have been added.


## Step 14: final locus list

In [65]:
import pandas as pd

# Load the data into a pandas DataFrame
data = pd.read_csv("11_pred_fisher_test_result.txt", sep="\t") #####change data set here############

# Remove rows with duplicate values in 'variable' column, while keeping the first occurrence
data_no_duplicates = data.drop_duplicates(subset='locus_id', keep='first')

# Save the modified DataFrames back to files
data_no_duplicates.to_csv('12_final_locus_data_pf_only_predscore_2.txt', sep='\t', index=False) #####change data set here############


## extra
this is for ranking

In [66]:
###ranking based on three methods

import pandas as pd

# Load the data into a pandas DataFrame
data = pd.read_csv("11_pred_fisher_test_result.txt", sep="\t") #####change data set here############

# Remove rows with duplicate values in 'variable' column, while keeping the first occurrence
data_no_duplicates = data.drop_duplicates(subset='locus_id', keep='first')

###---------------------------------------------------
# Sort the data based on PF_score
sorted_data = data_no_duplicates.sort_values(by="PF_score", ascending=False)

# Find and populate the rows where "known_effector" column is not empty
effectors_table = [["Ranking", "Known_effector", "p-value_lowest", "effector_score", "PF_score"]]
for i, (_, row) in enumerate(sorted_data.iterrows(), start=1):  # Using iterrows to iterate over DataFrame rows
    if pd.notnull(row["known_effector"]):  # Using pandas' notnull to check for non-empty values
        effectors_table.append([i, row["known_effector"], row["p-value_lowest"], row["effector_score"],row["PF_score"]])

# Convert the effectors_table list to a DataFrame
effectors_df = pd.DataFrame(effectors_table[1:], columns=effectors_table[0])

# Save the modified DataFrames back to files
data_no_duplicates.to_csv('12_final_locus_data_pf.txt', sep='\t', index=False) #####change data set here############
effectors_df.to_csv('13_ranking_PF.txt', sep='\t', index=False) #####change data set here############

###---------------------------------------------------
# Sort the data based on PF_score
sorted_data = data_no_duplicates.sort_values(by="effector_score", ascending=False)

# Find and populate the rows where "known_effector" column is not empty
effectors_table = [["Ranking", "Known_effector", "p-value_lowest", "effector_score", "PF_score"]]
for i, (_, row) in enumerate(sorted_data.iterrows(), start=1):  # Using iterrows to iterate over DataFrame rows
    if pd.notnull(row["known_effector"]):  # Using pandas' notnull to check for non-empty values
        effectors_table.append([i, row["known_effector"], row["p-value_lowest"], row["effector_score"],row["PF_score"]])

# Convert the effectors_table list to a DataFrame
effectors_df = pd.DataFrame(effectors_table[1:], columns=effectors_table[0])

# Save the modified DataFrames back to files
#data_no_duplicates.to_csv('12_final_locus_data_pf.txt', sep='\t', index=False) #####change data set here############
effectors_df.to_csv('13_ranking_pred_score.txt', sep='\t', index=False) #####change data set here############

###---------------------------------------------------
# Sort the data based on PF_score
sorted_data = data_no_duplicates.sort_values(by="p-value_lowest", ascending=False)

# Find and populate the rows where "known_effector" column is not empty
effectors_table = [["Ranking", "Known_effector", "p-value_lowest", "effector_score", "PF_score"]]
for i, (_, row) in enumerate(sorted_data.iterrows(), start=1):  # Using iterrows to iterate over DataFrame rows
    if pd.notnull(row["known_effector"]):  # Using pandas' notnull to check for non-empty values
        effectors_table.append([i, row["known_effector"], row["p-value_lowest"], row["effector_score"],row["PF_score"]])

# Convert the effectors_table list to a DataFrame
effectors_df = pd.DataFrame(effectors_table[1:], columns=effectors_table[0])

# Save the modified DataFrames back to files
data_no_duplicates.to_csv('12_final_locus_data_p-value.txt', sep='\t', index=False) #####change data set here############
effectors_df.to_csv('13_ranking_p-value.txt', sep='\t', index=False) #####change data set here############

